In [11]:
from stem import Signal
from stem.control import Controller
import requests
from stem.descriptor.remote import DescriptorDownloader
import random
import time

In [12]:
# Connect to the Tor process on the default port (9051)
controller = Controller.from_port(port=9051)

controller.authenticate(password='mKubavat1234')  # default password might be required if set

In [13]:
downloader = DescriptorDownloader()

try:
    consensus = downloader.get_consensus().run()
    all_relays = [relay for relay in consensus if 'Fast' and relay.flags and 'Running' in relay.flags]
finally:
    controller.close()

In [14]:
import random
from stem.control import Controller
from stem.descriptor.remote import DescriptorDownloader

# Establish a persistent Tor control connection
with Controller.from_port(port=9051) as controller:
    controller.authenticate(password='mKubavat1234')  # Use the correct password

    # Initialize DescriptorDownloader
    downloader = DescriptorDownloader()
    
    try:
        consensus = downloader.get_consensus().run()
        all_relays = [relay for relay in consensus if 'Fast' in relay.flags and 'Running' in relay.flags]

        def select_relay(relays):
            chosen = random.choice(relays)
            subnet = '.'.join(chosen.address.split('.')[:2])
            return chosen, subnet

        first, first_subnet = select_relay(all_relays)
        remaining_relays = [r for r in all_relays if not r.address.startswith(first_subnet)]
        second, second_subnet = select_relay(remaining_relays)
        remaining_relays = [r for r in remaining_relays if not r.address.startswith(second_subnet)]
        third, _ = select_relay(remaining_relays)

        # Attempt to build a circuit
        circuit_id = controller.new_circuit([first.fingerprint, second.fingerprint, third.fingerprint], await_build=True, timeout=240)
        print("Circuit established:", circuit_id)

    except Exception as e:
        print("An error occurred:", e)
    finally:
        controller.close()


Circuit established: 22


In [15]:
def request(url):
    proxies = {
        'http': 'socks5h://127.0.0.1:9050',
        'https': 'socks5h://127.0.0.1:9050'
    }
    response = requests.get(url, proxies=proxies)
    return response

start_time = time.time()
response = request('http://google.com')
elapsed_time = time.time() - start_time
print(f"Page loaded in {elapsed_time} seconds")

Page loaded in 1.0690104961395264 seconds
